In [ ]:
#QUESTION GENERATION

question_instructions = """You are a culinary analyst interviewing an expert about food/recipes.

Your goal is to gather specific, practical culinary insights.

1. Focus on: {goals}

2. Ask specific questions about:
   - Recipes and preparation methods
   - Nutritional information
   - Dietary modifications
   - Cooking techniques and tips

3. Be specific and practical

Begin by introducing yourself, then ask your question.

When satisfied, end with: "Thank you so much for your help!"
"""

def generate_question(state: InterviewState):
    analyst = state["analyst"]
    messages = state["messages"]

    system_message = question_instructions.format(goals=analyst.persona)
    question = llm.invoke([SystemMessage(content=system_message)] + messages)

    return {"messages": [question]}

print(" Question generation ready!")

In [ ]:
#WEB SEARCH FUNCTIONS

search_instructions = SystemMessage(content="""Generate a search query to find recipe and nutrition information.

Focus on:
- Recipe websites and food blogs
- Nutrition databases
- Cooking guides
- Dietary information

Create a precise food/recipe search query.""")

def search_web(state: InterviewState):
    structured_llm = llm.with_structured_output(SearchQuery)
    search_query = structured_llm.invoke([search_instructions] + state['messages'])

    print(f"       Searching web for: {search_query.search_query}")

    try:
        search_results = tavily_search.invoke(search_query.search_query)

        if isinstance(search_results, list):
            search_docs = search_results
        elif isinstance(search_results, dict):
            search_docs = search_results.get("results", [])
        else:
            search_docs = []

        if search_docs:
            # LIMIT CONTENT - Only first 500 chars per doc, max 3 docs
            formatted_search_docs = "\n\n---\n\n".join([
                f'<Document href="{doc.get("url", "N/A")}"/>\n{str(doc.get("content", doc.get("snippet", "")))[:500]}...\n</Document>'
                for doc in search_docs[:3]
                if isinstance(doc, dict)
            ])
            print(f"       Found {len(search_docs)} web results (truncated)")
        else:
            formatted_search_docs = "No search results found."
            print("       No web results found")

    except Exception as e:
        print(f"       Search error: {e}")
        formatted_search_docs = f"Search error occurred: {str(e)}"

    return {"context": [formatted_search_docs]}

def search_wikipedia(state: InterviewState):
    structured_llm = llm.with_structured_output(SearchQuery)
    search_query = structured_llm.invoke([search_instructions] + state['messages'])

    print(f"       Searching Wikipedia for: {search_query.search_query}")

    try:
        # LIMIT - Only 1 doc
        search_docs = WikipediaLoader(query=search_query.search_query, load_max_docs=1).load()

        if search_docs:
            # LIMIT CONTENT - Only first 1000 chars
            formatted_search_docs = "\n\n---\n\n".join([
                f'<Document source="{doc.metadata.get("source", "Wikipedia")}" page="{doc.metadata.get("page", "")}"/>\n{doc.page_content[:1000]}...\n</Document>'
                for doc in search_docs[:1]
            ])
            print(f"       Found{len(search_docs)} Wikipedia article (truncated)")
        else:
            formatted_search_docs = "No Wikipedia results found."
            print("       No Wikipedia results found")

    except Exception as e:
        print(f"       Wikipedia search error: {e}")
        formatted_search_docs = f"Wikipedia search error: {str(e)}"

    return {"context": [formatted_search_docs]}

print("✅ Web search functions ready (with token limits)!")